In [ ]:
import torch
from diffusers import DiffusionPipeline
from hy3dshape.pipelines import Hunyuan3DDiTFlowMatchingPipeline
from PIL import Image, ImageOps
from rembg import remove
import os
import warnings
import trimesh
import numpy as np

warnings.filterwarnings("ignore")


print("Инициализация моделей...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
shape_pipeline = Hunyuan3DDiTFlowMatchingPipeline.from_pretrained(
    'tencent/Hunyuan3D-2.1', torch_dtype=torch.float16
)
image_gen_pipeline = DiffusionPipeline.from_pretrained(
    "playgroundai/playground-v2.5-1024px-aesthetic", torch_dtype=torch.float16, variant="fp16"
).to(device)
print("Модели успешно инициализированы.")


def resize_and_pad(image: Image.Image, target_size: tuple[int, int], fill_color: tuple[int, int, int] = (255, 255, 255)) -> Image.Image:
    padded_image = Image.new("RGB", target_size, fill_color)
    image.thumbnail(target_size, Image.Resampling.LANCZOS)
    paste_position = ((target_size[0] - image.width) // 2, (target_size[1] - image.height) // 2)
    padded_image.paste(image, paste_position)
    return padded_image


def clean_mesh_by_largest_component(input_path: str, output_path: str) -> bool:
    try:
        mesh = trimesh.load_mesh(input_path, force='mesh')
        if not isinstance(mesh, trimesh.Trimesh): return False
        components = mesh.split(only_watertight=False)
        if len(components) <= 1:
            mesh.export(output_path)
            return True
        largest_component = max(components, key=lambda c: len(c.faces))
        largest_component.export(output_path)
        return True
    except Exception as e:
        print(f"Ошибка при очистке меша: {e}")
        return False


def decimate_mesh(input_path: str, output_path: str, target_face_count: int) -> bool:
    try:
        mesh = trimesh.load_mesh(input_path, force='mesh')
        if not isinstance(mesh, trimesh.Trimesh): return False
        if len(mesh.faces) <= target_face_count:
            mesh.export(output_path)
            return True
        new_mesh = mesh.simplify_quadric_decimation(face_count=target_face_count)
        new_mesh.export(output_path)
        return True
    except Exception as e:
        print(f"Ошибка при упрощении меша: {e}")
        return False


def process_generated_mesh(raw_path, output_prefix, low_poly_faces):
    cleaned_path = f"{output_prefix}_1_cleaned_high.obj"
    final_low_poly_path = f"{output_prefix}_2_final_low.obj"
    if not clean_mesh_by_largest_component(raw_path, cleaned_path): return
    if not decimate_mesh(cleaned_path, final_low_poly_path, target_face_count=low_poly_faces): return


def generate_mesh_from_text(
    prompt: str,
    output_prefix: str,
    low_poly_faces: int = 1500,
    seed: int = 42,
    num_inference_steps: int = 50,
    guidance_scale: float = 3.5
):
    try:
        output_dir = os.path.dirname(output_prefix)
        if not os.path.exists(output_dir): os.makedirs(output_dir)
        raw_path = f"{output_prefix}_0_raw.obj"

        generator = torch.Generator(device=device).manual_seed(seed)
        
        image = image_gen_pipeline(
            prompt=prompt,
            num_inference_steps=num_inference_steps,
            guidance_scale=guidance_scale,
            generator=generator
        ).images[0]

        image_no_bg_rgba = remove(image)
        white_background = Image.new("RGB", image_no_bg_rgba.size, (255, 255, 255))
        white_background.paste(image_no_bg_rgba, (0, 0), image_no_bg_rgba)
        
        image_on_white = white_background
        image_on_white.save(f"{output_prefix}_reference_image.png")
        
        processed_image = resize_and_pad(image_on_white, (512, 512))
        
        generated_mesh = shape_pipeline(image=processed_image, show_progress_bar=True)[0]
        generated_mesh.export(raw_path)
        
        process_generated_mesh(raw_path, output_prefix, low_poly_faces)

    except Exception as e:
        print(f"Произошла глобальная ошибка: {e}")
    print(f"Генерация для '{output_prefix}' завершена.")


def generate_mesh_from_image(image_path: str, output_prefix: str, low_poly_faces: int = 1500):
    print("\n" + "="*50)
    print(f"Начало генерации из изображения: {image_path}")
    if not os.path.exists(image_path):
        print(f"!!! Ошибка: Файл не найден {image_path}")
        return
    try:
        output_dir = os.path.dirname(output_prefix)
        if not os.path.exists(output_dir): os.makedirs(output_dir)
        raw_path = f"{output_prefix}_0_raw.obj"

        print("Обработка референсного изображения...")
        image = Image.open(image_path).convert("RGB")
        processed_image = resize_and_pad(image, (512, 512))
        
        print("Генерация сырого 3D-меша...")
        generated_mesh = shape_pipeline(image=processed_image, show_progress_bar=True)[0]
        generated_mesh.export(raw_path)
        
        process_generated_mesh(raw_path, output_prefix, low_poly_faces)

    except Exception as e:
        print(f"!!! Произошла глобальная ошибка: {e}")
    print("Генерация завершена.")
    print("="*50 + "\n")


generate_mesh_from_text(
    prompt="(masterpiece), (best quality), game asset, a single longsword, front view, orthographic, 3d model, 3d render, hyper detailed, clean, ((white background)), ((isolated on white)), professional, studio lighting, sharp focus",
    output_prefix="../output/sword_imp_g",
    low_poly_faces=1000
)

2025-08-03 13:28:25,637 - hy3dgen.shapgen - INFO - Try to load model from local path: /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1
2025-08-03 13:28:25,643 - hy3dgen.shapgen - INFO - Loading model from /root/.cache/hy3dgen/tencent/Hunyuan3D-2.1/hunyuan3d-dit-v2-1/model.fp16.ckpt


Инициализация моделей...
using moe
using moe
using moe
using moe
using moe
using moe
PointCrossAttentionEncoder INFO: pc_sharpedge_size is zero


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

Модели успешно инициализированы.


  0%|          | 0/50 [00:00<?, ?it/s]

Volume Decoding: 100%|████████████████████████████████████████| 7134/7134 [00:13<00:00, 517.31it/s]


Генерация для '../output/sword_imp_g' завершена.


In [ ]:
generate_mesh_from_text(
    prompt="photorealistic 3d model of a mechanical dragon, steampunk style, single object, centered, uniform pure white background, no shadow, studio lighting, product shot, 8k",
    output_prefix="../output/dragon",
    low_poly_faces=2000
)

  0%|          | 0/50 [00:00<?, ?it/s]

Volume Decoding: 100%|████████████████████████████████████████| 7134/7134 [00:13<00:00, 512.02it/s]


Генерация для '../output/dragon' завершена.
